In [38]:
import torch
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import math
import numpy as np
print(torch.cuda.is_available())
print(torch.__version__)

True
2.2.1


In [3]:
# 导入模型
# windows
model = SentenceTransformer("D:\\code\\bert_model\\pre_trained\\distiluse-base-multilingual-cased-v1")
#mac
#model = SentenceTransformer("distiluse-base-multilingual-cased-v1")


#multilingual model
# distiluse-base-multilingual-cased-v1: Multilingual knowledge distilled version of multilingual Universal Sentence Encoder. Supports 15 languages: Arabic, Chinese, Dutch, English, French, German, Italian, Korean, Polish, Portuguese, Russian, Spanish, Turkish.

# distiluse-base-multilingual-cased-v2: Multilingual knowledge distilled version of multilingual Universal Sentence Encoder. This version supports 50+ languages, but performs a bit weaker than the v1 model.

# paraphrase-multilingual-MiniLM-L12-v2 - Multilingual version of paraphrase-MiniLM-L12-v2, trained on parallel data for 50+ languages.

# paraphrase-multilingual-mpnet-base-v2 - Multilingual version of paraphrase-mpnet-base-v2, trained on parallel data for 50+ languages.

In [4]:
#函数定义
def split_dataset(time_start,time_end,df):
    return df.loc[time_start:time_end]

def get_split_dfs(start,end,diff,exp_data):
    dfs = []
    ends = start + diff -1
    year = start
    n = int((end - start)/diff)
    for i in range(n):
        time_start = f'{start}-01-01'
        time_end = f'{ends}-12-31'
        # print(time_start)
        dfs.append(split_dataset(time_start,time_end,exp_data))
        start += diff
        ends += diff
    for i in dfs:
        print(f"{year}:{len(i)}")
        year += diff
    return dfs

In [8]:
#数据处理
#国家年度专利数量统计
all_data = pd.read_excel('all_data.xlsx')
all_data.set_index(pd.to_datetime(all_data["申请日"]),inplace=True)
all_data.sort_index(inplace=True)
all_dfs = get_split_dfs(2010,2020,2,all_data)

2010:32761
2012:33471
2014:34880
2016:34737
2018:31522


In [61]:
#处理实验国别
#['公开（公告）号','公开（公告）日','申请号','申请日','公开国别','标准化申请人','申请人国家/地区','标题 (中文)','标题 (英文)','摘要 (中文)','摘要 (英文)','标题（小语种原文）','摘要（小语种原文）']
exp_code = {'USA': 0, 'GBR': 1, 'MYS': 2, 'ITA': 3, 'AUS': 4, 'FRA': 5, 'PHL': 6, 'CAN': 7, 'BRB': 8, 'JPN': 9, 'FIN': 10, 'CHN': 11, 'DEU': 12, 'AUT': 13, 'TWN': 14, 'IND': 15, 'SGP': 16, 'NLD': 17, 'KOR': 18, 'SWE': 19, 'CHE': 20, 'BEL': 21, 'ISR': 22, 'VGB': 23, 'CYM': 24, 'NOR': 25, 'IRL': 26, 'ESP': 27, 'RUS': 28, 'NZL': 29, 'DNK': 30, 'HKG': 31}
df = pd.read_excel('CH-EN_countries.xlsx')
exp_countries = exp_code.keys()
ch_to_en = {}
en_to_ch = {}
for index,row in df.iterrows():
    en_to_ch[row['三位字母']] = row['中文名']
    ch_to_en[row['中文名']] = row['三位字母']

empty = []
for i in range(len(exp_countries)):
    empty.append([])
corpus_list = []


for i in range(len(all_dfs)):
    corpus = dict(zip(exp_countries,empty))
    for index,row in all_dfs[i].iterrows():
        for i in row['申请人国家/地区'].split(', '):
            code = ch_to_en[i]
            if code in exp_countries:
                if not pd.isna(row['摘要（小语种原文）']):
                    cur_corpus = row['摘要（小语种原文）'] 
                elif code in ['CHN', 'TWN', 'HKG']:
                    cur_corpus = row['摘要 (中文)']
                else:
                    cur_corpus = row['摘要 (英文)']
                corpus[code].append(cur_corpus)
        corpus_list.append(corpus)

            

In [ ]:
sentences = [
    "纪欢航在门外",
    "Jihuanhang is at outside of the door",
    "不想玩啦！",
    "纪欢航在花园"
]
sentence_embeddings = model.encode(sentences)
matrix = util.cos_sim(sentence_embeddings,sentence_embeddings)

print(matrix)